In [1]:
import Base: getindex, size, println
function println(x)
    show(STDOUT, "text/plain", x)
    print("\n")
end
using BenchmarkTools
using Compat.Test

In [2]:
########## Bit-Wise Operations ##############
const BInt = Int64
const DInt = Int64

basis(num_bit::Int) = UnitRange{DInt}(0, 1<<num_bit-1)

########## BitArray views ###################
import Base: BitArray
function bitarray(v::Vector{T}) where T<:Number
    xdim = sizeof(eltype(v))*8
    #ba = BitArray{2}(0, 0)
    ba = BitArray(0, 0)
    ba.chunks = reinterpret(UInt64, v)
    ba.dims = (xdim, length(v))
    ba.len = xdim*length(v)
    return ba
end

function bitarray(v::Vector{UInt64})
    #ba = BitArray{2}(0, 0)
    ba = BitArray(undef, (0, 0))
    ba.chunks = v
    ba.dims = (64, length(v))
    ba.len = 64*length(v)
    return ba
end
bitarray(v::Number) = bitarray([v])

bitarray (generic function with 3 methods)

In [3]:
# bit size
bsizeof(x) = sizeof(x) << 3
function bit_length(x::DInt)
    local n = 0
    while x!=0
        n += 1
        x >>= 1
    end
    return n
end

function log2i(x::T)::T where T
    local n::T = 0
    while x&0x1!=1
        n += 1
        x >>= 1
    end
    return n
end

log2i (generic function with 1 method)

In [4]:
const Ints = Union{Vector{Int}, Int, UnitRange{Int}}
const DInts = Union{Vector{DInt}, DInt, UnitRange{DInt}}
bmask(ibit::Int...) = reduce(+, [one(DInt) << b for b in (ibit.-1)])
bmask(bits::UnitRange{Int}) = ((one(DInt) << (bits.stop - bits.start + 1)) - one(DInt)) << (bits.start-1)

# take a bit/bits
takebit(index::DInt, ibit::Int) = (index >> (ibit-1)) & 1
# a position is 1?
testbit(index::DInt, mask::Int) = (index & mask) != 0

# set a bit
setbit(index::DInt, mask::Int) = indices | mask
setbit!(indices::DInts, mask::Int) = indices[:] |= mask

# flip a bit/bits
flip(index::DInt, mask::Int) = index ⊻ mask
flip!(indices::DInt, mask::Int) = indices[:] = indices .⊻ mask
# flip all bits
flip(index::DInt) = ~index
flip!(indices::DInts) = indices[:] = ~indices  #can not be scalar

flip! (generic function with 2 methods)

In [2]:
"""move 1st dim to last"""
function lrollaxis2!(vec::AbstractArray)
    n = length(vec)
    halfn = n >> 1
    temp = vec[2:2:n]
    vec[1:halfn] = vec[1:2:n]
    vec[halfn+1:end] = temp
    vec
end

"""move last dim to 1st"""
function rrollaxis2!(vec::AbstractArray)
    n = length(vec)
    halfn = n >> 1
    temp = vec[halfn+1:end]
    vec[1:2:n] = vec[1:halfn]
    vec[2:2:n] = temp
    vec
end

function rollaxis2!(v::AbstractArray, k::Int)
    if k > 0
        for i=1:k
            lrollaxis2!(v)
        end
    else
        for i=1:-k
            rrollaxis2!(v)
        end
    end
    v
end

rollaxis2! (generic function with 1 method)

In [3]:
v = randn([2 for i=1:10]...)
v2 = copy(v)
@assert rollaxis2!(v, 5) == permutedims(v2, [(i+4)%10+1 for i=1:10])
@assert rollaxis2!(v, -5) == v2
@assert rollaxis2!(v, 0) == v2

In [4]:
@benchmark permutedims(v2, [(i)%10+1 for i=1:10])

BenchmarkTools.Trial: 
  memory estimate:  48.94 KiB
  allocs estimate:  2573
  --------------
  minimum time:     79.902 μs (0.00% GC)
  median time:      83.254 μs (0.00% GC)
  mean time:        93.313 μs (2.52% GC)
  maximum time:     2.020 ms (87.37% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [5]:
@benchmark rollaxis2!(v2, 1)

BenchmarkTools.Trial: 
  memory estimate:  8.45 KiB
  allocs estimate:  11
  --------------
  minimum time:     2.056 μs (0.00% GC)
  median time:      2.206 μs (0.00% GC)
  mean time:        2.685 μs (12.72% GC)
  maximum time:     121.815 μs (91.77% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [6]:
function roll_prod(s, n, ops)
    st = reshape(s, 2, 1<<(n-1))

    for i = 1:n
        st .= ops[i] * st
        rollaxis2!(s)
    end
    s
end

roll_prod (generic function with 1 method)

In [7]:
function focus_prod(s, n, ops)
    st = reshape(s, 2, 1<<(n-1))
    st .= first(ops) * st

    for i = 2:n
        s = reshape(s, ntuple(x->2, Val{n}))
        perm = collect(1:n)
        perm[1] = i
        perm[i] = 1
        permutedims!(s, s, perm)
        st = reshape(s, 2, 1<<(n-1))
        st .= ops[i] * st
        permutedims!(s, s, perm)
    end
    s
end

focus_prod (generic function with 1 method)

In [8]:
ops = [rand(2, 2) for i=1:10];
s = rand(1<<10)
n = 10
@benchmark focus_prod($s, n, $ops)

BenchmarkTools.Trial: 
  memory estimate:  855.78 KiB
  allocs estimate:  49321
  --------------
  minimum time:     1.581 ms (0.00% GC)
  median time:      1.609 ms (0.00% GC)
  mean time:        1.736 ms (2.14% GC)
  maximum time:     3.806 ms (32.90% GC)
  --------------
  samples:          2874
  evals/sample:     1

In [9]:
ops = [rand(2, 2) for i=1:10];
s = rand(1<<10)
n = 10
@benchmark roll_prod($s, n, $ops)

LoadError: [91mMethodError: no method matching rollaxis2!(::Array{Float64,1})[0m
Closest candidates are:
  rollaxis2!(::AbstractArray, [91m::Int64[39m) at In[2]:22[39m

In [32]:
bss = [1,2,3,6,7]
println([bin(b,4) for b in bss])

5-element Array{String,1}:
 "0001"
 "0010"
 "0011"
 "0110"
 "0111"


In [47]:
# swap two bits
function swapbits(num::DInt, i::Int, j::Int)
    i = i-1
    j = j-1
    k = (num >> j) & 1 - (num >> i) & 1
    num + k*(1<<i) - k*(1<<j)
end
function swapbits!(bss::Vector{DInt}, i::Int, j::Int)
    @views bss[:] = swapbits.(bss, i, j)
    bss
end
swapbits(5, 1,2)

6

In [48]:
swapbits!(bss, 1,2)
println([bin(b,4) for b in bss])

5-element Array{String,1}:
 "0010"
 "0001"
 "0011"
 "0101"
 "0111"


In [49]:
rolling([X, Y, (), chain(K2,G2), L])

LoadError: [91mUndefVarError: rolling not defined[39m

In [55]:
i,j=2,1
b=collect(1:1000)
swapbits.(b, i,j)
#@benchmark swapbits.($b, $i, $j)

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     10.705 μs (0.00% GC)
  median time:      13.204 μs (0.00% GC)
  mean time:        14.663 μs (2.21% GC)
  maximum time:     490.105 μs (87.81% GC)
  --------------
  samples:          10000
  evals/sample:     3

In [57]:
bin(bmask(1:5))

"11111"

In [69]:
# utils used in controled bits
function indices_with1(num_bit::Int, poss::Vector{Int}, vals::Vector{BInt})
    sls = Vector{Union{Colon, Int}}([Colon() for i=1:num_bit])
    sls[poss] = vals.+1
    reshape(getindex(reshape(basis(num_bit), fill(2, num_bit)...), sls...), 1<<(num_bit-length(poss)))
end

indices_with1 (generic function with 1 method)

In [70]:
num_bit = 16
#indices_with1(num_bit, [3, 7], [1, 1])
@benchmark indices_with1(num_bit, [3, 7], [1, 1])

BenchmarkTools.Trial: 
  memory estimate:  78.15 MiB
  allocs estimate:  1082322
  --------------
  minimum time:     208.441 ms (4.25% GC)
  median time:      214.902 ms (5.53% GC)
  mean time:        223.924 ms (5.14% GC)
  maximum time:     320.309 ms (4.36% GC)
  --------------
  samples:          23
  evals/sample:     1

In [91]:
neg(index::DInt, num_bit::Int) = bmask(1:num_bit) ⊻ index
@show bmask(1,2,3])
bin(neg(bmask(1,2,3), 16))

bmask(1, 2, 3) = 7


"1111111111111000"

In [130]:
function indices_with(num_bit::Int, poss::Vector{Int}, vals::Vector{BInt})
    mask = bmask(poss...)
    valmask = bmask(poss[vals.!=0]...)
    filter(i->i.&mask.==valmask, basis(num_bit))
end

indices_with (generic function with 2 methods)

In [131]:
num_bit = 16
@test indices_with(num_bit, [1,2,3,4,5,6], [0,0,1,1,0,1]) == indices_with1(num_bit, [1,2,3,4,5,6], [0,0,1,1,0,1])
@benchmark indices_with(num_bit, [7,3], [1,1])

BenchmarkTools.Trial: 
  memory estimate:  197.69 KiB
  allocs estimate:  33
  --------------
  minimum time:     128.805 μs (0.00% GC)
  median time:      129.897 μs (0.00% GC)
  mean time:        148.638 μs (5.49% GC)
  maximum time:     2.476 ms (88.45% GC)
  --------------
  samples:          10000
  evals/sample:     1